In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
#from sklearn.model_selection import cross_val_score
from scipy.io import mmread
import numpy as np

In [3]:
malware_classes = ["Agent", "AutoRun", "FraudLoad", "FraudPack", "Hupigon", "Krap",
           "Lipler", "Magania", "None", "Poison", "Swizzor", "Tdss",
           "VB", "Virut", "Zbot"]

# a function for writing predictions in the required format
def write_predictions(predictions, ids, outfile):
    """
    assumes len(predictions) == len(ids), and that predictions[i] is the
    index of the predicted class with the malware_classes list above for
    the executable corresponding to ids[i].
    outfile will be overwritten
    """
    with open(outfile,"w+") as f:
        # write header
        f.write("Id,Prediction\n")
        for i, history_id in enumerate(ids):
            f.write("%s,%d\n" % (history_id, predictions[i]))

def classes_to_Y(classes):
    output = []
    for cls in classes:
        output.append(malware_classes.index(cls))
    return np.array(output)

In [4]:
# load training classes
classes = np.load("../data/features/train_classes.npy")

# load sparse matrix of training data
sparse_mat_train_test = mmread("../data/features/naive_word_hashed_full_features.mtx")

In [9]:
# convert csr to a numpy array
sparse = sparse_mat_train_test

# pull out training examples
X = sparse.tocsc()[:classes.shape[0],:]
# X_CV = X[-300:]
# X = X[:-300]

X_test = sparse.tocsc()[classes.shape[0]:,:]
print X_test.shape

Y = classes_to_Y(classes)
# Y_CV = Y[-300:]
# Y = Y[:-300]

(3724, 1048576)


In [6]:
RF = RandomForestClassifier()
RF.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [1]:
# preds = RF.predict(X_CV)

# mistakes = 0
# for i in range(len(preds)):
#     if preds[i] != Y_CV[i]:
#         mistakes += 1
#         print malware_classes[Y_CV[i]]
# print mistakes

NameError: name 'RF' is not defined

In [ ]:
test_pred = RF.predict(X_test)

In [ ]:
print test_pred
test_ids = np.load("../data/features/test_ids.npy")
print test_ids
write_predictions(test_pred, test_ids, "../predictions/rfc_10.csv")

In [26]:
# Try an SGDClassifier learning directly from the sparse matrix
SGD = SGDClassifier(n_jobs=4, n_iter=100, verbose=1, loss="modified_huber")
SGD.fit(X,Y)

-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 49.63, NNZs: 137576, Bias: -1.086566, T: 3086, Avg. loss: 2.054229
Total training time: 0.04 seconds.
Norm: 42.51, NNZs: 117300, Bias: -0.964139, T: 3086, Avg. loss: 0.589489
Total training time: 0.03 seconds.
Norm: 34.32, NNZs: 40677, Bias: -1.280956, T: 3086, Avg. loss: 0.662277
Total training time: 0.04 seconds.
-- Epoch 2
-- Epoch 2
Norm: 42.92, NNZs: 21922, Bias: -1.209116, T: 3086, Avg. loss: 0.709926
Total training time: 0.05 seconds.
-- Epoch 2
-- Epoch 2
Norm: 28.25, NNZs: 122808, Bias: -1.027954, T: 6172, Avg. loss: 0.360902
Total training time: 0.06 seconds.
Norm: 24.86, NNZs: 53736, Bias: -1.449923, T: 6172, Avg. loss: 0.431927
Total training time: 0.07 seconds.
-- Epoch 3
-- Epoch 3
Norm: 31.27, NNZs: 24436, Bias: -1.284387, T: 6172, Avg. loss: 0.441692
Total training time: 0.07 seconds.
-- Epoch 3
Norm: 20.91, NNZs: 65237, Bias: -1.527439, T: 9258, Avg. loss: 0.324200
Total training time: 0.10 seconds.
Norm: 38.23, NNZs: 1

[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:   14.0s finished


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=100,
       n_jobs=4, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, verbose=1, warm_start=False)

In [27]:
test_pred = SGD.predict(X_test)

In [28]:
print test_pred
test_ids = np.load("../data/features/test_ids.npy")
print test_ids
write_predictions(test_pred, test_ids, "../predictions/sgd_huber.csv")

[10  8  8 ..., 10  8  8]
['298379101ad17b816ffeb88e8ba9ce915b7a46fc9'
 'cb56d8fd4accec76a5608a000d65711d1cfa9cc3e'
 '21ead3c92f31d801b1a887c1b0dc12d503f7204d6' ...,
 'E96d65723d24aa91a9fa54f92754444f99491d2e1'
 '107e0ba3d61f2650b09ef02a30f0bab0b82c1933d'
 '16a64eeb54Ca49adf9608e2206db5aedc22038a09']


In [17]:
from sklearn.linear_model import PassiveAggressiveClassifier
# Try an SGDClassifier learning directly from the sparse matrix
model = PassiveAggressiveClassifier(n_jobs=4, verbose=42, n_iter=20)
model.fit(X,Y)

-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 7.55, NNZs: 125613, Bias: -0.138381, T: 3086, Avg. loss: 0.046595
Total training time: 0.03 seconds.
Norm: 5.78, NNZs: 164319, Bias: -0.112992, T: 3086, Avg. loss: 0.120186
Total training time: 0.04 seconds.
-- Epoch 2
-- Epoch 2
Norm: 5.25, NNZs: 75659, Bias: -0.112442, T: 3086, Avg. loss: 0.035022
Total training time: 0.04 seconds.
-- Epoch 2
Norm: 9.22, NNZs: 132801, Bias: -0.179926, T: 6172, Avg. loss: 0.041148
Total training time: 0.06 seconds.
Norm: 5.13, NNZs: 144209, Bias: -0.137939, T: 3086, Avg. loss: 0.042037
Total training time: 0.06 seconds.
-- Epoch 3
-- Epoch 2
Norm: 8.61, NNZs: 182575, Bias: -0.142782, T: 6172, Avg. loss: 0.116928
Total training time: 0.07 seconds.
-- Epoch 3
Norm: 11.13, NNZs: 138897, Bias: -0.200858, T: 9258, Avg. loss: 0.038695Norm: 10.86, NNZs: 194803, Bias: -0.189226, T: 9258, Avg. loss: 0.114809
Total training time: 0.12 seconds.

Total training time: 0.12 seconds.
-- Epoch 4
Norm: 7.00, NNZs: 1515

[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.9s


-- Epoch 4
Norm: 5.63, NNZs: 135322, Bias: -0.115125, T: 3086, Avg. loss: 0.036669
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 7.53, NNZs: 171739, Bias: -0.145867, T: 3086, Avg. loss: 0.043442
Total training time: 0.04 seconds.
-- Epoch 2Norm: 23.85, NNZs: 178723, Bias: -0.321923, T: 61720, Avg. loss: 0.019125
Norm: 11.52, NNZs: 80865, Bias: -0.244189, T: 12344, Avg. loss: 0.039950

Total training time: 0.84 seconds.Total training time: 0.11 seconds.

-- Epoch 5
-- Epoch 1
Norm: 7.56, NNZs: 152761, Bias: -0.179906, T: 6172, Avg. loss: 0.032229
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 13.14, NNZs: 82053, Bias: -0.285887, T: 15430, Avg. loss: 0.038661
Total training time: 0.14 seconds.
Norm: 8.83, NNZs: 158628, Bias: -0.208310, T: 9258, Avg. loss: 0.029973
Total training time: 0.08 seconds.
-- Epoch 6
-- Epoch 4
Norm: 10.25, NNZs: 188905, Bias: -0.233700, T: 6172, Avg. loss: 0.035793
Total training time: 0.08 seconds.
Norm: 6.44, NNZs: 110961, Bias: -0.124772, T: 3086,

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    1.6s


Norm: 21.27, NNZs: 192523, Bias: -0.768249, T: 49376, Avg. loss: 0.017036
Total training time: 0.62 seconds.
-- Epoch 20
-- Epoch 16
-- Epoch 17
Norm: 17.26, NNZs: 179921, Bias: 0.054885, T: 3086, Avg. loss: 0.379791
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 23.05, NNZs: 181732, Bias: -0.640700, T: 61720, Avg. loss: 0.021388
Total training time: 0.65 seconds.
Norm: 21.75, NNZs: 192523, Bias: -0.794348, T: 52462, Avg. loss: 0.016664
Total training time: 0.65 seconds.
Norm: 21.61, NNZs: 121756, Bias: -0.495703, T: 49376, Avg. loss: 0.022391
Total training time: 0.60 seconds.
-- Epoch 18
-- Epoch 1
-- Epoch 17
Norm: 24.26, NNZs: 199880, Bias: 0.092845, T: 6172, Avg. loss: 0.338764
Total training time: 0.06 seconds.
-- Epoch 3
Norm: 22.15, NNZs: 193170, Bias: -0.824890, T: 55548, Avg. loss: 0.016280
Total training time: 0.67 seconds.
Norm: 22.25, NNZs: 122093, Bias: -0.507598, T: 52462, Avg. loss: 0.021986
Total training time: 0.63 seconds.
-- Epoch 19
-- Epoch 18
Norm: 3.98, NNZ

...

Norm: 19.15, NNZs: 126304, Bias: -0.257922, T: 58634, Avg. loss: 0.017318
Total training time: 0.79 seconds.
-- Epoch 20
Norm: 16.99, NNZs: 120622, Bias: -0.288076, T: 3086, Avg. loss: 0.190011
Total training time: 0.04 seconds.
-- Epoch 2
Norm: 33.16, NNZs: 202317, Bias: -0.428273, T: 55548, Avg. loss: 0.025384
Total training time: 0.73 seconds.
-- Epoch 19
Norm: 23.18, NNZs: 168430, Bias: -0.708947, T: 46290, Avg. loss: 0.026629
Total training time: 0.70 seconds.
-- Epoch 16
Norm: 19.79, NNZs: 126370, Bias: -0.260907, T: 61720, Avg. loss: 0.017141
Total training time: 0.83 seconds.
-- Epoch 1
Norm: 21.49, NNZs: 124552, Bias: -0.345108, T: 6172, Avg. loss: 0.150839
Total training time: 0.08 seconds.
-- Epoch 3
Norm: 33.54, NNZs: 202500, Bias: -0.431745, T: 58634, Avg. loss: 0.024636
Total training time: 0.79 seconds.
Norm: 23.89, NNZs: 168732, Bias: -0.739428, T: 49376, Avg. loss: 0.026162
Total training time: 0.75 seconds.
Norm: 4.65, NNZs: 112943, Bias: -0.063152, T: 3086, Avg. loss

[Parallel(n_jobs=4)]: Done  16 out of  15 | elapsed:    2.5s remaining:   -0.2s


Norm: 19.52, NNZs: 130246, Bias: -0.058023, T: 33946, Avg. loss: 0.052161
Total training time: 0.38 seconds.
Norm: 40.74, NNZs: 142850, Bias: -0.703940, T: 40118, Avg. loss: 0.086770
Total training time: 0.46 seconds.
-- Epoch 12
Norm: 13.23, NNZs: 40818, Bias: -0.087900, T: 24688, Avg. loss: 0.022145
Total training time: 0.31 seconds.
-- Epoch 14
-- Epoch 9
Norm: 20.80, NNZs: 131729, Bias: -0.064334, T: 37032, Avg. loss: 0.051542
Total training time: 0.42 seconds.
Norm: 41.85, NNZs: 142958, Bias: -0.716650, T: 43204, Avg. loss: 0.084812
Total training time: 0.49 seconds.
-- Epoch 13
Norm: 13.53, NNZs: 40933, Bias: -0.092750, T: 27774, Avg. loss: 0.021465
Total training time: 0.34 seconds.
-- Epoch 15
-- Epoch 10
Norm: 21.88, NNZs: 131825, Bias: -0.077474, T: 40118, Avg. loss: 0.050901
Total training time: 0.45 seconds.
Norm: 42.89, NNZs: 143206, Bias: -0.749313, T: 46290, Avg. loss: 0.082909
Total training time: 0.53 seconds.
-- Epoch 14
Norm: 14.24, NNZs: 40966, Bias: -0.093801, T: 3

[Parallel(n_jobs=4)]: Done  16 out of  15 | elapsed:    2.9s remaining:   -0.2s
[Parallel(n_jobs=4)]: Done  16 out of  15 | elapsed:    3.0s remaining:   -0.2s
[Parallel(n_jobs=4)]: Done  16 out of  15 | elapsed:    3.1s remaining:   -0.2s
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:    3.1s finished


PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='hinge', n_iter=20, n_jobs=4, random_state=None,
              shuffle=True, verbose=42, warm_start=False)

In [18]:
test_pred = model.predict(X_test)

In [19]:
print test_pred
test_ids = np.load("../data/features/test_ids.npy")
print test_ids
write_predictions(test_pred, test_ids, "../predictions/passive_basic.csv")

[10  8  8 ..., 10  8  8]
['298379101ad17b816ffeb88e8ba9ce915b7a46fc9'
 'cb56d8fd4accec76a5608a000d65711d1cfa9cc3e'
 '21ead3c92f31d801b1a887c1b0dc12d503f7204d6' ...,
 'E96d65723d24aa91a9fa54f92754444f99491d2e1'
 '107e0ba3d61f2650b09ef02a30f0bab0b82c1933d'
 '16a64eeb54Ca49adf9608e2206db5aedc22038a09']
